In [1]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
!pip install -U torchtext
!pip install git+https://github.com/facebookresearch/detectron2.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-vdrp08i0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-vdrp08i0
  Resolved https://github.com/facebookresearch/detectron2.git to commit 536dc9d527074e3b15df5f6677ffe1f4e104a4ab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.8 M

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sunnykusawa/tshirts")

print("Path to dataset files:", path)

100%|██████████| 358M/358M [00:03<00:00, 109MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sunnykusawa/tshirts/versions/2


In [3]:
import os
folder = os.listdir(path + '/tshirt')
folder

#path + '/tshirt/'1652.jpg''

['723.jpg',
 '1340.jpg',
 '417.jpg',
 '519.jpg',
 '1736.jpg',
 '1116.jpg',
 '1393.jpg',
 '265.jpg',
 '74.jpg',
 '1143.jpg',
 '1063.jpg',
 '212.jpg',
 '1141.jpg',
 '169.jpg',
 '725.jpg',
 '872.jpg',
 '456.jpg',
 '1935.jpg',
 '159.jpg',
 '641.jpg',
 '687.jpg',
 '476.jpg',
 '1047.jpg',
 '1430.jpg',
 '1303.jpg',
 '914.jpg',
 '1367.jpg',
 '990.jpg',
 '165.jpg',
 '803.jpg',
 '1955.jpg',
 '382.jpg',
 '1669.jpg',
 '890.jpg',
 '41.jpg',
 '272.jpg',
 '1379.jpg',
 '943.jpg',
 '1104.jpg',
 '1043.jpg',
 '860.jpg',
 '1421.jpg',
 '1484.jpg',
 '965.jpg',
 '780.jpg',
 '483.jpg',
 '1609.jpg',
 '1405.jpg',
 '1963.jpg',
 '1155.jpg',
 '1516.jpg',
 '523.jpg',
 '1735.jpg',
 '1691.jpg',
 '1070.jpg',
 '1873.jpg',
 '899.jpg',
 '1636.jpg',
 '1727.jpg',
 '1581.jpg',
 '1984.jpg',
 '1646.jpg',
 '1322.jpg',
 '811.jpg',
 '836.jpg',
 '1388.jpg',
 '1687.jpg',
 '765.jpg',
 '192.jpg',
 '1983.jpg',
 '1465.jpg',
 '1750.jpg',
 '1631.jpg',
 '1656.jpg',
 '1695.jpg',
 '38.jpg',
 '630.jpg',
 '1181.jpg',
 '919.jpg',
 '1649.jpg',

In [6]:
import torch
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from sklearn.cluster import KMeans

# Initialize the model once to reuse it across function calls
def setup_predictor():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.DEVICE = "cpu"
    return DefaultPredictor(cfg)

predictor = setup_predictor()

# Extract dominant color from a masked object
def extract_dominant_color(image, mask, k=3):
    masked_pixels = image[mask == 1]
    if masked_pixels.size == 0:
        return None
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(masked_pixels)
    dominant_color = kmeans.cluster_centers_[np.argmax(np.bincount(kmeans.labels_))]
    return dominant_color.astype(int)

# Main function
def get_dominant_colors(image_bgr):
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    outputs = predictor(image_rgb)
    instances = outputs["instances"].to("cpu")
    masks = instances.pred_masks.numpy()

    dominant_colors = []
    for mask in masks:
        color = extract_dominant_color(image_rgb, mask)
        if color is not None:
            dominant_colors.append(color.tolist())  # Convert to list for easier JSON/log output

    return dominant_colors


model_final_f10217.pkl: 178MB [00:01, 143MB/s]                           


In [10]:
image_path = path  +'/tshirt/1125.jpg'
image = cv2.imread(image_path)
colors = get_dominant_colors(image)
print("Dominant RGB colors:", colors)


Dominant RGB colors: [[195, 15, 44]]
